# CharNN

## Convert pitches to characters and words

In [1]:
import converters.convert as cvt
import numpy as np
stream1 = cvt.text_to_pitch_stream('corpus1.txt')
stream2 = cvt.text_to_pitch_stream('corpus2.txt')
stream3 = cvt.text_to_pitch_stream('corpus3.txt')
stream = stream1+stream2+stream3

In [2]:
import re
def convertToText(stream):
    """Converts stream of instruments to char text"""
    single_stream = [e for s in stream for e in s] # Lazy way of making single stream
    single_stream = set(single_stream)
    offset = 33 #To avoid unprintable characters
    pitch_chr_dict = dict([(pitch, chr(offset+i)) for i,pitch in enumerate(single_stream)])
    chr_pitch_dict = dict([(chr(offset+i), pitch) for i,pitch in enumerate(single_stream)])
    corpus = ''.join([pitch_chr_dict[e] for s in stream for e in s])
    corpus = re.sub(r'(!){2}', '', corpus) # Remove silence notes
    return single_stream, pitch_chr_dict, chr_pitch_dict, corpus

In [3]:
single_stream, pitch_chr_dict, chr_pitch_dict, corpus = convertToText(stream)

In [4]:
with open('text.txt', 'w') as f:
    f.write(corpus)

In [5]:
# Assumes that out of vocab pitch does not appear in the conditioning set
n_letters = len(single_stream)

def letterToIndex(letter):
    """returns index of the letter in the dictionary"""
    return ord(letter) - offset

def letterToTensor(letter):
    """Converts the given letter to tensor"""
    tensor = np.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

def wordToTensor(word):
    """Converts a word to a list of tensors"""
    tensor = np.zeros(len(word), 1, n_letters)
    for wi, letter in enumerate(word):
        tensor[wi][0][letterToIndex(letter)] = 1
    return tensor

def pitchToTensor(array):
    """Converts pitch array to one hot tensors"""
    tensor = [letterToTensor(pitch_chr_dict[l]) for s in array for l in s]
    tensor = np.stack(tensor)
    tensor = np.reshape(tensor, (*array.shape, -1))
    return tensor
    

In [6]:
def get_batches(sequence, batch_size, seq_length=4):
    """Generator that returns batches of size batch size x total length of sequence
    
    args:
    ----
    sequence: input text corpus
    batch_size: number of sequences per batch
    seq_length: number of characters per sequence. Default 4 for 4 instruments
    """
    batch_size_total = batch_size*seq_length
    n_batches = len(sequence)//batch_size_total
    sequence = sequence[:n_batches*batch_size_total] # throw away remainder
    sequence = sequence.reshape((batch_size, -1))
    
    for n in range(0, sequence.shape[1], seq_length):
        # The features
        x = sequence[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], sequence[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], sequence[:, 0]
        yield x, y 

Testing

In [7]:
encoded = np.array([chr_pitch_dict[c] for c in corpus])
batches = get_batches(encoded, 8)
x, y = next(batches)

In [8]:
print(x)
print(y)

[[67 67  0 55]
 [ 0 61  0 64]
 [64 52  0 64]
 [ 0 72 60  0]
 [ 0 56  0 68]
 [74  0 62 59]
 [79  0 48  0]
 [79  0 79  0]]
[[67  0 55 67]
 [61  0 64  0]
 [52  0 64  0]
 [72 60  0 60]
 [56  0 68  0]
 [ 0 62 59 74]
 [ 0 48  0 78]
 [ 0 79  0 79]]


In [9]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


with io.open('text.txt', encoding='utf-8') as f:
    text = f.read()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars)), dropout=0.1))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

model.fit(x, y,
          batch_size=64,
          epochs=10)

Using TensorFlow backend.


corpus length: 1235770
total chars: 73
nb sequences: 411890
Vectorization...
Build model...
Epoch 1/10
411890/411890 [==============================] - 628s 2ms/step - loss: 2.2391
Epoch 2/10
411890/411890 [==============================] - 609s 1ms/step - loss: 2.0507
Epoch 3/10
411890/411890 [==============================] - 516s 1ms/step - loss: 2.0170
Epoch 4/10
411890/411890 [==============================] - 1067s 3ms/step - loss: 1.9968
Epoch 5/10
411890/411890 [==============================] - 1362s 3ms/step - loss: 1.9863
Epoch 6/10
411890/411890 [==============================] - 1420s 3ms/step - loss: 1.9770
Epoch 7/10
411890/411890 [==============================] - 2904s 7ms/step - loss: 1.9727
Epoch 8/10
411890/411890 [==============================] - 1393s 3ms/step - loss: 1.9908
Epoch 9/10
411890/411890 [==============================] - 3477s 8ms/step - loss: 1.9877
Epoch 10/10
411890/411890 [==============================] - 1416s 3ms/step - loss: 1.9715


In [10]:
with open('F.txt') as f:
    bach = f.read()
_, _, _, condition = convertToText(bach)
for index,diversity in enumerate([0.2, 0.5, 1.0, 1.2]):

    predicted_text = []
    for i in range(10000):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(condition[-maxlen:]):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                condition = condition[1:] + next_char
                predicted_text.append(next_char)

    predicted_text = ''.join(predicted_text) 
    with open('predictions'+str(index)+'.txt', 'w') as f:
        print('writing predictions'+str(index)+'.txt')
        f.write(predicted_text)

writing predictions0.txt
writing predictions1.txt
writing predictions2.txt
writing predictions3.txt


## Convert text back to melody

In [11]:
import glob
for file in glob.glob('predictions*.txt'):
    print('reading', file)
    with open(file, 'r') as f:
        notes = f.read()
    
    pitches = np.array([chr_pitch_dict[c] for c in notes])
    pitches = pitches.reshape((-1, 4))
    mid = cvt.pitchstream_to_midi(pitches)
    mid.save(file+'.mid')


reading predictions.txt
reading predictions0.txt
reading predictions1.txt
reading predictions2.txt
reading predictions3.txt
